# 単回帰分析
2021年：1世帯（二人以上の世帯）当たり　年間収入 と 1か月支出

## データ収集の方法
家計調査結果，総務省統計局　http://www.stat.go.jp/data/kakei/index.htm    
#### 方法１：
サイト:https://www.e-stat.go.jp/内で "二人以上の世帯" "年間収入五分位・十分位階級別" "2021"を検索<br>
#### 方法2:
サイト　https://www.e-stat.go.jp/　内の[分野] &rarr;   企業・家計・経済内の[家計調査] &rarr; [ファイル]　&rarr; [家計収支偏]－[二人以上世帯]－[年報]－[年次]　&rarr; [2021年度]　 <br>

方法1，2と共に，最後に現れるWebページで，「年間収入五分位・十分位階級別」－「二人以上の世帯・勤労者世帯」この右欄にあるEXCELをクリックしてCSVファイルをダウンロード。
このExcelファイル内で, 年間収入十分位階級から， <br> 
- 年間収入(万円)，各階級の下限値 -> "income"  
- 消費支出 -> "expenditure" 

と表現してデータを作成。ただし、階級Iは0～270万円であり、この幅が大きすぎるため，適当に値を定めた。この値はデータを参照されたい。<br>

## statsmodelsの単回帰分析の使用方法
- 線形回帰モデルの使用例　http://www.statsmodels.org/stable/regression.html  
- OLS（Ordinary Least Squares）の説明　http://www.statsmodels.org/stable/examples/notebooks/generated/ols.html  
- R風の引数の与え方　http://www.statsmodels.org/dev/example_formulas.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline

### データの読み込み
df_origは後に利用するため。<br>
df.corr()は，各変数間の相関を見るため。

In [ ]:
df = pd.read_csv('dataFamilyBudget2021.csv', comment='#')
df_orig = df.copy() # deep copy to keep original data
print(df)
print(df.shape)
df.corr()

### OLS
最小2乗)法を用いて単回帰モデルの係数を求める：https://en.wikipedia.org/wiki/Ordinary_least_squares<br>
データ数 < 20 の場合には、尖度（kurtosis）の評価（olsではkurtosistestと表記）が有効にできないというwarningメッセージが現れるが、ここではこの評価は使わないので無視する。<br>
scipy.stats.kurtosistest https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.kurtosistest.html <br>
Kurtosis: https://en.wikipedia.org/wiki/Kurtosis

In [ ]:
result = smf.ols('expenditure ~ income', data=df).fit() # 引数：R風の書き方
print(result.summary())

次のWariningが出た場合"UserWarning: kurtosistest only valid for n>=20"，これは尖度（kurtosis）の計算にはデータ数(n)は20以上必要，と言っている。この例では，尖度は用いないので無視する。

#### 用語
- Dep. Variable: dependent variable，直訳は従属変数であるが，目的変数のこと
- Model: OLS	Ordinary Least Squares
- Medhod: Least Squares, 最小二乗法を適用
- No. Observations: 観測数（データ数）
- R-squared: 決定係数$R^2$
- Adj. R-squared: Adjusted coefficient of determination,  調整済み決定係数
- F-statistic,  F検定量
- AIC: Akaike information criterion

Intercept($\beta_0$), income($\beta_1$) の表において
- coef: coefficient, 係数の値
- std err: standard error, 係数の標準偏差
- t: $T$検定量
- P>|t|: tの値に対する確率
- [0.025 0.975]: 95\% 信頼区間

#### t検定量の計算，次式と同じか否かの検証
下記のTb0, Tb1が上記の表のtと同じ値であることを確認する。

In [ ]:
b0, b1 = result.params # 偏回帰係数の取り出し方
print('b0=',b0,'  b1=',b1) 
x = df['income'].to_numpy()
y = df['expenditure'].to_numpy()
N = len(x)
meanx = np.mean(x)
Sxx = np.sum((x-meanx)**2)
xsqr  = np.sum(x**2)
esqr = (y - (b0 + b1*x))**2 # 残差の平方, Squares of residuals
Ssqr = np.sum(esqr)/(N-2)# 残差の平方和

In [ ]:
Tb0 = b0/np.sqrt(Ssqr*xsqr/(N*Sxx))
Tb1 = b1/np.sqrt(Ssqr/Sxx)
print('Tb0=',Tb0,' Tb1=',Tb1)

上記の結果とstatsmodelsの計算結果を比べて，同じ計算を行っていることがわかる。

#### 散布図と単回帰モデル

In [ ]:
plt.plot(df['income'],df['expenditure'], 'o')
plt.plot(df['income'], b0+b1*df['income'], c='k')

plt.xlabel('income [x 10000YEN]/Year')
plt.ylabel('expenditure [YEN]/Month')

#plt.tight_layout()
plt.grid()
#plt.savefig('REG_Simple_FamilyInEx01.pdf', bbox_inches='tight')
plt.show()

#### 予測

In [ ]:
NewData = {'income':[900,1200]}
dfnew = pd.DataFrame(NewData)
pred = result.predict(dfnew)
pred

次は同じことを計算している

In [ ]:
b0 + b1*1200

In [ ]:
fig = plt.subplots(figsize=(5,3))
plt.plot(df['income'],df['expenditure'], 'o', color='white', markeredgecolor='k')
#plt.plot(df['income'], b0+b1*df['income'], c='k')

plt.plot(dfnew.income[0],pred[0], 'o', color='k')
plt.plot(dfnew.income[1],pred[1], 'o', color='k')

xrange= np.arange(100,1300, 10)
plt.plot(xrange, b0+b1*xrange, c='k')

plt.xlabel('income [x 10000YEN]/Year')
plt.ylabel('expenditure [YEN]/Month')

plt.grid()
#plt.savefig('REG_Simple_FamilyIncomeExpenditure.pdf', bbox_inches='tight')
plt.show()